In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import numpy.typing as npt
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import json
import itertools

2023-05-03 20:55:46.498683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# defining constants
TEST_SIZE: np.float_ = .3
RANDOM_STATE: np.int_ = 42

In [3]:
# importing and handling data
data: pd.DataFrame = pd.read_csv("data/input/data_tp1", header=None)
data.iloc[:, data.columns != 0] = data.iloc[:, data.columns != 0] / 255
data.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def MNIST_MLP(data: npt.NDArray[np.int_], hidden_layer_size: np.int_, batch_size: np.int_, learning_rate: np.int_) -> dict:

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(784,)),
        tf.keras.layers.Dense(hidden_layer_size, activation="sigmoid"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

    input_data: npt.NDArray[np.int_] = data[:, 1:]
    labels: npt.NDArray[np.int_] = data[:, 0]
    X_train, X_test, y_train, y_test = train_test_split(input_data, labels, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    epochs: np.int_ = 10

    model_history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=0)

    y_pred: npt.NDArray[np.int_] = model.predict(X_test, batch_size=batch_size, verbose=0)
    y_pred = y_pred.argmax(axis=-1)

    accuracy_score: np.float_ = sklearn.metrics.accuracy_score(y_test, y_pred)
    precision_score: npt.NDArray[np.float_] = sklearn.metrics.precision_score(y_test, y_pred, average=None, zero_division=0)
    recall_score: npt.NDArray[np.float_] = sklearn.metrics.recall_score(y_test, y_pred, average=None, zero_division=0)
    f1_score: npt.NDArray[np.float_] = sklearn.metrics.f1_score(y_test, y_pred, average=None)
    confusion_matrix: npt.NDArray[np.int_] = sklearn.metrics.confusion_matrix(y_test, y_pred)

    run_info: dict = {
        "accuracy_score": accuracy_score,
        "precision_score": precision_score.tolist(),
        "recall_score": recall_score.tolist(),
        "f1_score": f1_score.tolist(),
        "confusion_matrix": confusion_matrix.tolist(),
        "random_state_seed": RANDOM_STATE,
        "test_size": TEST_SIZE,
        "hidden_layer_size": hidden_layer_size,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "history": model_history.history
    }

    return run_info

In [5]:
hidden_layer_sizes: npt.NDArray[np.int_] = np.array([(25,), (50,), (100,)])
batch_sizes: list[int] = [1, 20, 50, 3500]
lerning_rates: npt.NDArray[np.float_] = np.array([0.5, 1.0, 10.0])

configurations: list = list(itertools.product(hidden_layer_sizes, batch_sizes, lerning_rates))

run_infos: list[dict] = [MNIST_MLP(data=data.to_numpy(), hidden_layer_size=a, batch_size=b, learning_rate=c) for a, b, c in configurations]

2023-05-03 20:55:48.868794: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 20:55:48.869557: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
with open("data/results.json", "a") as file:
    json.dump([run_info for run_info in run_infos], file, indent=4)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/lfrf/anaconda3/envs/MNIST-MLP/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
  File "/tmp/ipykernel_57371/4031375013.py", line 2, in <module>
    json.dump([run_info for run_info in run_infos], file, indent=4)
    ^^^^
NameError: name 'json' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lfrf/anaconda3/envs/MNIST-MLP/lib/python3.11/site-packages/pygments/styles/__init__.py", line 78, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lfrf/anaconda3/envs/MNIST-MLP/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
  File "/home/lfrf/anaconda3/envs/MNIST-MLP/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1310, in

In [ ]:
df = pd.read_json("data/results.json")
df.nsmallest(n=10, columns=["accuracy_score"])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/lfrf/anaconda3/envs/MNIST-MLP/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
  File "/tmp/ipykernel_57371/2262362849.py", line 1, in <module>
    df = pd.read_json("data/results.json")
         ^^
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lfrf/anaconda3/envs/MNIST-MLP/lib/python3.11/site-packages/pygments/styles/__init__.py", line 78, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lfrf/anaconda3/envs/MNIST-MLP/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
  File "/home/lfrf/anaconda3/envs/MNIST-MLP/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
  

De acordo com https://www.tensorflow.org/guide/core/mlp_core, Next, rescale the data so that the pixel values of [0,255] fit into the range of [0,1]. This step ensures that the input pixels have similar distributions and helps with training convergence.